In [30]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [13]:
df=pd.read_csv("Fertilizer Prediction.csv")
df.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,26,52,35,Black,Wheat,14,10,36,DAP
1,34,65,38,Clayey,Pulses,33,0,0,Urea
2,25,50,35,Sandy,Tobacco,21,0,10,28-28
3,29,53,39,Black,Oil seeds,13,10,10,20-20
4,28,54,33,Clayey,Pulses,31,0,0,Urea


In [14]:
df.describe()

,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,30.314800,59.297900,43.374500,19.883600,1.304400,18.364300
std,3.559996,5.195138,10.676172,11.126519,3.306786,14.115917
min,25.000000,50.000000,28.000000,7.000000,0.000000,0.000000
25%,27.000000,54.000000,34.000000,11.000000,0.000000,0.000000
50%,30.000000,60.000000,42.000000,15.000000,0.000000,19.000000
75%,34.000000,64.000000,51.000000,29.000000,0.000000,30.000000
max,36.000000,68.000000,63.000000,41.000000,17.000000,40.000000


In [15]:
df.isnull().sum()

Temparature        0
Humidity           0
Moisture           0
Soil Type          0
Crop Type          0
Nitrogen           0
Potassium          0
Phosphorous        0
Fertilizer Name    0
dtype: int64

## Preprocessing

In [16]:
soil_encoder = LabelEncoder()
crop_encoder = LabelEncoder()
fertilizer_encoder = LabelEncoder()

df['Soil Type'] = soil_encoder.fit_transform(df['Soil Type'])
df['Crop Type'] = crop_encoder.fit_transform(df['Crop Type'])
df['Fertilizer Name'] = fertilizer_encoder.fit_transform(df['Fertilizer Name'])

df.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,26,52,35,0,9,14,10,36,5
1,34,65,38,1,6,33,0,0,6
2,25,50,35,4,8,21,0,10,4
3,29,53,39,0,4,13,10,10,3
4,28,54,33,1,6,31,0,0,6


## Train test Split

In [17]:
X=df.drop(['Fertilizer Name'],axis=1)
y=df['Fertilizer Name']

In [18]:
sc=StandardScaler()
X=sc.fit_transform(X)

In [19]:
from sklearn.model_selection import RepeatedStratifiedKFold
from tpot import TPOTClassifier

In [26]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = TPOTClassifier(generations=5, population_size=100, cv=cv, scoring='accuracy', verbosity=2, random_state=1, n_jobs=-1)
model.fit(X, y)

                                                                               
                                                                               
TPOT closed during evaluation in one generation.
                                                                               
                                                                               
TPOT closed prematurely. Will use the current best pipeline.
                                                                               
Best pipeline: XGBClassifier(input_matrix, learning_rate=0.5, max_depth=1, min_child_weight=8, n_estimators=100, n_jobs=1, subsample=0.7500000000000001, verbosity=0)


TPOTClassifier(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=1),
               generations=5, n_jobs=-1, random_state=1, scoring='accuracy',
               verbosity=2)

In [28]:
final_model = model.fitted_pipeline_
print(final_model)

Pipeline(steps=[('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.5,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=1, max_leaves=None, min_child_weight=8,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=100, n_jobs=1,
                  

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
final_model.fit(X_train, y_train)

Pipeline(steps=[('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.5,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=1, max_leaves=None, min_child_weight=8,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=100, n_jobs=1,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [32]:
y_pred = final_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 80.65%
